AVCAD - Exercise 6

Students: Maria Dolgaya (28168) and Mariana Coelho (25605)

In this exercise you will use again the dataset in EFIplus_medit.zip to perform some hypothesis testing
1. Using the EFIplus_medit.zip dataset Test if the frequency of sites with presence and absence of Salmo
trutta fario (Brown Trout) are independent from the country. Please state which is/are the null
hypothesis of your test(s).
2. Run the non-parametric equivalent of the test you used in exercise 5.3 and compare with the ANOVA
test (5.2: Test whether there are diferences in the mean elevation in the upstream catchment
(Elevation_mean_catch) among the eight most sampled catchments. For which pairs of catchments are
these diferences significant? Please state which is/are the null hypothesis of your test(s)).
3. Using the winequality_red.csv file in the examples folder of the github repository, test which wine
parameters discriminate the best between wine quality scores categorized into two classes using value
5 as the threshold value (quality>5=“good” and quality<5=“bad”).

In [2]:
import numpy as np
import pandas as pd
import scipy.stats as sts
import statsmodels.stats as stm
from statsmodels.formula.api import ols
import scikit_posthocs as sp
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Load the dfset
df = pd.read_csv('EFIplus_medit.zip',compression='zip', sep=";")

In [5]:
df

,Site_code,Latitude,Longitude,Country,Catchment_name,Galiza,Subsample,Calib_EFI_Medit,Calib_connect,Calib_hydrol,...,Squalius malacitanus,Squalius pyrenaicus,Squalius torgalensis,Thymallus thymallus,Tinca tinca,Zingel asper,Squalius sp,Barbatula sp,Phoxinus sp,Iberochondrostoma_sp
0,ES_01_0002,38.102003,-4.096070,Spain,Guadalquivir,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,ES_02_0001,40.530188,-1.887796,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,ES_02_0002,40.595432,-1.928079,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,ES_02_0003,40.656184,-1.989831,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,ES_02_0004,40.676402,-2.036274,Spain,Tejo,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5006,PTVO60,40.641612,-8.325149,Portugal,Vouga,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5007,PTVO61,40.729015,-8.098851,Portugal,Vouga,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5008,PTVO62,40.727959,-8.107344,Portugal,Vouga,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
5009,PTVO63,40.817783,-8.224903,Portugal,Vouga,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
list(df.columns)

['Site_code',
 'Latitude',
 'Longitude',
 'Country',
 'Catchment_name',
 'Galiza',
 'Subsample',
 'Calib_EFI_Medit',
 'Calib_connect',
 'Calib_hydrol',
 'Calib_morphol',
 'Calib_wqual',
 'Geomorph1',
 'Geomorph2',
 'Geomorph3',
 'Water_source_type',
 'Flow_regime',
 'Altitude',
 'Geological_typology',
 'Actual_river_slope',
 'Natural_sediment',
 'Elevation_mean_catch',
 'prec_ann_catch',
 'temp_ann',
 'temp_jan',
 'temp_jul',
 'Barriers_catchment_down',
 'Barriers_river_segment_up',
 'Barriers_river_segment_down',
 'Barriers_number_river_segment_up',
 'Barriers_number_river_segment_down',
 'Barriers_distance_river_segment_up',
 'Barriers_distance_river_segment_down',
 'Impoundment',
 'Hydropeaking',
 'Water_abstraction',
 'Hydro_mod',
 'Temperature_impact',
 'Velocity_increase',
 'Reservoir_flushing',
 'Sedimentation',
 'Channelisation',
 'Cross_sec',
 'Instream_habitat',
 'Riparian_vegetation',
 'Embankment',
 'Floodprotection',
 'Floodplain',
 'Toxic_substances',
 'Acidification',
 '

In [6]:
#1 Using the EFIplus_medit.zip dataset Test if the frequency of sites with presence and absence of Salmo trutta fario (Brown Trout) are independent from the country. Please state which is/are the null hypothesis of your test(s).
table = pd.crosstab(df['Country'], df['Salmo trutta fario'])
table

Salmo trutta fario,0,1
Country,,
France,13,59
Italy,109,76
Portugal,615,252
Spain,1239,2648


In [41]:
# H0: the frequency of sites with presence and absence of Salmo trutta fario (Brown Trout) are independent from the country

stat, p, df, expected_freq = sts.chi2_contingency(table)
print('Statistics=%.3f, p=%.6f' % (stat, p))
alpha=0.05
if p > alpha:
 print('fail to reject H0. Rejecting H0 has an error probability >0.05')
else:
 print('reject H0 with an error probability <0.05)')

Statistics=496.372, p=0.000000
reject H0 with an error probability <0.05)


In [7]:
#2 Run the non-parametric equivalent of the test you used in exercise 5.3 and compare with the ANOVA test (5.2: Test whether there are diferences in the mean elevation in the upstream catchment (Elevation_mean_catch) among the eight most sampled catchments. For which pairs of catchments are these diferences significant? Please state which is/are the null hypothesis of your test(s)).
catchment_count = pd.crosstab(index = df['Catchment_name'], columns='count')
print(catchment_count)

col_0                  count
Catchment_name              
Algarve-Sotavento          6
Algarve-barlavento        10
Arno-Ombrone               2
Campania-South            19
Cantabrica               502
Catala                   242
Corse                      5
Douro                    401
Ebro                     736
EntreDouroeVouga           2
Galiza-Norte             709
Galiza-Sul                95
Garonne                    3
Guadalquivir              63
Guadia                   313
Italia-Veneto              1
Jucar                    103
Jucar-Ebro                52
Minho                    707
Mira                       5
Mondego                   86
Odelouca                  40
Oeste                     33
Ombrone                    5
Ombrone-Tevere             6
Rhone                     16
Sado                       9
Saline-Pescara-Sangro     29
Sardinia                  17
Segura                    42
Segura-Jucar               7
South_France_E            23
South_France_W

In [8]:
top_catch = df[df['Catchment_name'].isin(['Minho', 'Tejo', 'Galiza-Norte', 'Ebro', 'Cantabrica', 'Douro', 'Guadia', 'Catala'])]
data = top_catch[['Catchment_name', 'Elevation_mean_catch']].dropna()
data

,Catchment_name,Elevation_mean_catch
1,Tejo,1603.519424
2,Tejo,1578.678579
3,Tejo,1553.219128
4,Tejo,1539.684999
5,Tejo,1493.914010
...,...,...
4947,Tejo,704.675069
4948,Tejo,702.673201
4949,Tejo,697.730192
4950,Tejo,679.784324


In [10]:
presence = df[df['Salmo trutta fario'] == 1]
absence = df[df['Salmo trutta fario'] == 0]
presence_temp = presence['temp_ann'].dropna()
absence_temp = absence['temp_ann'].dropna()

# Compute the test
stat, p = sts.kruskal(presence_temp, absence_temp)
print('F-statistics=%.3f, p=%.6f' % (stat, p))
alpha=0.05
if p > alpha:
 print('fail to reject H0. Rejecting H0 has an error probability >0.05')
else:
 print('reject H0 with an error probability <0.05)')

F-statistics=1383.760, p=0.000000
reject H0 with an error probability <0.05)


In [9]:
# 3 Using the winequality_red.csv file in the examples folder of the github repository, test which wine parameters discriminate the best between wine quality scores categorized into two classes using value 5 as the threshold value (quality>5=“good” and quality<5=“bad”).

df_wine = pd.read_csv('winequality_red.csv')
df_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [10]:
good = df_wine.loc[df_wine['quality'] > 5, 'quality']
bad = df_wine.loc[df_wine['quality'] <= 5, 'quality']

In [49]:
# H0: The two samples are drawn from populations with equal distributions

stat, p = sts.ks_2samp(good, bad)
print('stat=%.3f, p-value=%.3f' % (stat, p))
alpha=0.05
if p > alpha:
 print('fail to reject H0. Rejecting H0 has an error probability >0.05')
else:
 print('reject H0 with an error probability <0.05)')

stat=1.000, p-value=0.000
reject H0 with an error probability <0.05)


In [50]:
stat, p = sts.mannwhitneyu(good, bad, alternative='two-sided')
print('stat=%.3f, p-value=%.3f' % (stat, p))
alpha=0.05
if p > alpha:
 print('fail to reject H0. Rejecting H0 has an error probability >0.05')
else:
 print('reject H0 with an error probability <0.05)')

stat=636120.000, p-value=0.000
reject H0 with an error probability <0.05)
